In [2]:
!pip install transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 5.8 MB 39.1 MB/s 
     |████████████████████████████████| 182 kB 83.4 MB/s 
     |████████████████████████████████| 7.6 MB 61.7 MB/s 


In [6]:
#Importar librerías

import pandas as pd
import torch
from torch.utils.data import DataLoader, RandomSampler, SequentialSampler
from torch.utils.data import TensorDataset

import transformers
from transformers import BertTokenizer
from transformers import BertForSequenceClassification
from transformers import AdamW, get_linear_schedule_with_warmup

from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score


import nltk
from nltk.corpus import stopwords


import random
import numpy as np
from tqdm import tqdm

# To see errors in GPU in case it happens
import os
os.environ['CUDA_LAUNCH_BLOCKING'] = "1"

nltk.download('stopwords')


import re

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [35]:
#Lectura de datos

df = pd.read_csv("train.csv")
df_submission = pd.read_csv("test.csv")
input_key = "text"
label_key = "name"

stop_words = set(stopwords.words('spanish'))

In [36]:
#Aplicar transformaciones: eliminar tildes, quitar mayúsculas, eliminar stopwords

a,b = 'áéíóúüñÁÉÍÓÚÜÑ','aeiouunAEIOUUN'
trans=str.maketrans(a,b)

#df["text"]=df["text"].apply(lambda x:x.lower())
df["text"]=df["text"].apply(lambda x:x.translate(trans))
#df["text"]=df["text"].str.split(' ').apply(lambda x: ' '.join(k for k in x if k not in stop_words))
df["text"]=df["text"].apply(lambda x: re.split('https:\/\/.*', str(x))[0])



#df_submission["text"]=df_submission["text"].apply(lambda x:x.lower())
df_submission["text"]=df_submission["text"].apply(lambda x:x.translate(trans))
#df_submission["text"]=df_submission["text"].str.split(' ').apply(lambda x: ' '.join(k for k in x if k not in stop_words))
df_submission["text"]=df_submission["text"].apply(lambda x: re.split('https:\/\/.*', str(x))[0])




In [ ]:
df

,id,name,text
0,bad072c3848858f8e311d43b,Petro,es aqui con la belleza que comienza el torrent...
1,b4d1292e78f6bf6eb1d6afdf,Petro,preparando el encuentro del pacto en popayan.
2,65fa3e93c11cd6bbd20b768a,Petro,este es uno de los mayores errores en obras pu...
3,bb630a5e1f83acaaad90f8ce,Petro,para ser justos este es el producto del proyec...
4,b2b4fd1e629b3383d222ab94,Petro,"en santiago de guambia, cauca. reunido con la ..."
...,...,...,...
5533,8d4a044952357d0cca72b717,Lopez,"¡en este #viernessanto nos seguimos cuidando, ..."
5534,083344c6689d45419d02e3ea,Lopez,"bogota, recuerden que hasta el lunes 5 de abri..."
5535,3d59496a6ca65de2b5843491,Lopez,#buenasnoticias | 232 mil personas han salido ...
5536,f6aa036c136b9b726432a982,Lopez,"#ensemanasantanoscuidamos, por eso recuerden q..."


In [37]:
#Eliminar duplicados
print(df.shape)
df = df.drop_duplicates("id").reset_index(drop=True)
print(df.shape)

df = df[[input_key, label_key]]

(5538, 3)
(5538, 3)


In [38]:
#Tomar la variable de etiquetas

possible_labels = df[label_key].unique()

label_dict = {}
for index, possible_label in enumerate(possible_labels):
    label_dict[possible_label] = index

In [39]:
#Codificar las etiquetas

df['label'] = df[label_key].replace(label_dict)

In [ ]:
label_dict

{'Petro': 0, 'Uribe': 1, 'Lopez': 2}

In [40]:
#Dividir en entrenamiento y test
df = df[[input_key, "label"]]

X_train, X_val, y_train, y_val = train_test_split(df.index.values, 
                                                  df.label.values, 
                                                  test_size=0.1, 
                                                  random_state=42, 
                                                  stratify=df.label.values)

df['data_type'] = ['not_set']*df.shape[0]

df.loc[X_train, 'data_type'] = 'train'
df.loc[X_val, 'data_type'] = 'val'
print(df[df['data_type'] == "train"]["label"].nunique())
print(df[df['data_type'] == "val"]["label"].nunique())

a = set(df[df['data_type'] == "train"]["label"].unique())
b = set(df[df['data_type'] == "train"]["label"].unique())

print(len(a), len(b), len(a.intersection(b)))

3
3
3 3 3


In [13]:
#Qué tan largos son los tweets

seq_len=[len(i.split()) for i in train_text]

pd.Series(seq_len).hist(bins=200)

NameError: ignored

In [69]:
#Definir el tokenizador usando BETO:

tokenizer = BertTokenizer.from_pretrained('dccuchile/bert-base-spanish-wwm-uncased')
                                     
encoded_data_train = tokenizer.batch_encode_plus(
    df[df.data_type=='train'][input_key].values, 
    add_special_tokens=True, 
    return_attention_mask=True, 
    pad_to_max_length=True, 
    max_length=45, 
    return_tensors='pt',
    truncation=True
)

encoded_data_val = tokenizer.batch_encode_plus(
    df[df.data_type=='val'][input_key].values, 
    add_special_tokens=True, 
    return_attention_mask=True, 
    pad_to_max_length=True, 
    max_length=45, 
    return_tensors='pt',
    truncation=True
)


input_ids_train = encoded_data_train['input_ids']
attention_masks_train = encoded_data_train['attention_mask']
labels_train = torch.tensor(df[df.data_type=='train'].label.values)

input_ids_val = encoded_data_val['input_ids']
attention_masks_val = encoded_data_val['attention_mask']
labels_val = torch.tensor(df[df.data_type=='val'].label.values)

dataset_train = TensorDataset(input_ids_train, attention_masks_train, labels_train)
dataset_val = TensorDataset(input_ids_val, attention_masks_val, labels_val)

In [70]:
#Pasar parámetros al tokenizador

encoded_data_submission = tokenizer.batch_encode_plus(
    df_submission[input_key].values,
    add_special_tokens=True, 
    return_attention_mask=True, 
    pad_to_max_length=True, 
    max_length=45, 
    return_tensors='pt',
    truncation=True
)

input_ids_submission = encoded_data_submission['input_ids']
attention_masks_submission = encoded_data_submission['attention_mask']

dataset_submission = TensorDataset(input_ids_submission, attention_masks_submission)

In [71]:
## Definir modelo

model = BertForSequenceClassification.from_pretrained("dccuchile/bert-base-spanish-wwm-uncased",
                                                      num_labels=len(label_dict),
                                                      output_attentions=False,
                                                      output_hidden_states=False)

# model.parameters

Some weights of the model checkpoint at dccuchile/bert-base-spanish-wwm-uncased were not used when initializing BertForSequenceClassification: ['cls.predictions.decoder.weight', 'cls.predictions.decoder.bias', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.dense.weight']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at dccuc

In [77]:
#Definir otros parámetros y métricas de evaluación

batch_size = 16

dataloader_train = DataLoader(dataset_train, 
                              sampler=RandomSampler(dataset_train), 
                              batch_size=batch_size)

dataloader_validation = DataLoader(dataset_val, 
                                   sampler=SequentialSampler(dataset_val), 
                                   batch_size=batch_size)



optimizer = AdamW(model.parameters(),
                  lr=1e-5, 
                  eps=1e-8)
                  
epochs = 4

scheduler = get_linear_schedule_with_warmup(optimizer, 
                                            num_warmup_steps=0,
                                            num_training_steps=len(dataloader_train)*epochs)



def f1_score_func(preds, labels):
    preds_flat = np.argmax(preds, axis=1).flatten()
    labels_flat = labels.flatten()
    return f1_score(labels_flat, preds_flat, average='weighted')

def accuracy_per_class(preds, labels):
    label_dict_inverse = {v: k for k, v in label_dict.items()}
    
    preds_flat = np.argmax(preds, axis=1).flatten()
    labels_flat = labels.flatten()

    for label in np.unique(labels_flat):
        y_preds = preds_flat[labels_flat==label]
        y_true = labels_flat[labels_flat==label]
        print(f'Class: {label_dict_inverse[label]}')
        print(f'Accuracy: {len(y_preds[y_preds==label])}/{len(y_true)}\n')

def total_accuracy(preds, labels):
    
    preds_flat = np.argmax(preds, axis=1).flatten()
    labels_flat = labels.flatten()

    y_preds = preds_flat
    y_true = labels_flat

    return np.sum(y_preds==y_true)/len(y_true)


In [78]:
#Dataloader para datos de test

dataloader_submission = DataLoader(dataset_submission, 
                                   sampler=SequentialSampler(dataset_submission), 
                                   batch_size=batch_size)


In [79]:
#Usar GPU y definir semilla

seed_val = 17
random.seed(seed_val)
np.random.seed(seed_val)
torch.manual_seed(seed_val)
torch.cuda.manual_seed_all(seed_val)
torch.cuda.empty_cache()

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model.to(device)
print(device)

cuda


In [80]:
#Función de evaluación

def evaluate(dataloader_val):

    model.eval()
    
    loss_val_total = 0
    predictions, true_vals = [], []
    
    for batch in dataloader_val:

        batch = tuple(b.to(device) for b in batch)
        # batch = tuple(b for b in batch)

        
        inputs = {'input_ids':      batch[0],
                  'attention_mask': batch[1],
                  'labels':         batch[2],
                 }

        with torch.no_grad():        
            outputs = model(**inputs)
            
        loss = outputs[0]
        logits = outputs[1]
        loss_val_total += loss.item()

        logits = logits.detach().cpu().numpy()
        label_ids = inputs['labels'].cpu().numpy()
        predictions.append(logits)
        true_vals.append(label_ids)
    
    loss_val_avg = loss_val_total/len(dataloader_val) 
    
    predictions = np.concatenate(predictions, axis=0)
    true_vals = np.concatenate(true_vals, axis=0)
            
    return loss_val_avg, predictions, true_vals

In [81]:
#Entrenamiento

for epoch in tqdm(range(1, epochs+1)):

    model.train()
    
    loss_train_total = 0

    progress_bar = tqdm(dataloader_train, desc='Epoch {:1d}'.format(epoch), leave=False, disable=False)
    for batch in progress_bar:


        model.zero_grad()
        
        batch = tuple(b.to(device) for b in batch)
        # batch = tuple(b for b in batch)

        inputs = {'input_ids':      batch[0],
                  'attention_mask': batch[1],
                  'labels':         batch[2],
                 }
       

        outputs = model(**inputs)
        
        loss = outputs[0]
        loss_train_total += loss.item()
        loss.backward()

        torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)

        optimizer.step()
        scheduler.step()
        
        progress_bar.set_postfix({'training_loss': '{:.3f}'.format(loss.item()/len(batch))}, refresh=False)
         
        
    torch.save(model.state_dict(), f'/content/finetuned_BERT_epoch_{epoch}.model')
        
    tqdm.write(f'\nEpoch {epoch}')
    
    loss_train_avg = loss_train_total/len(dataloader_train)            
    tqdm.write(f'Training loss: {loss_train_avg}')
    
    val_loss, predictions, true_vals = evaluate(dataloader_validation)
    val_total_acc = total_accuracy(predictions, true_vals)
    tqdm.write(f'Validation loss: {val_loss}')
    tqdm.write(f'total_accuracy: {val_total_acc}')
    tqdm.write("------")


  0%|          | 0/4 [01:07<?, ?it/s]


Epoch 1
Training loss: 0.3631948826309198


 25%|██▌       | 1/4 [01:09<03:27, 69.23s/it]

Validation loss: 0.45195517273885866
total_accuracy: 0.8321299638989169
------



 25%|██▌       | 1/4 [02:17<03:27, 69.23s/it]

...

Validation loss: 0.6915874226018787
total_accuracy: 0.8411552346570397
------



 75%|███████▌  | 3/4 [04:35<01:09, 69.41s/it]


Epoch 4
Training loss: 0.07048992151864625


100%|██████████| 4/4 [04:37<00:00, 69.40s/it]

Validation loss: 0.6979322303518919
total_accuracy: 0.851985559566787
------


In [82]:
## Evaluacion

torch.cuda.empty_cache()


model = BertForSequenceClassification.from_pretrained("dccuchile/bert-base-spanish-wwm-uncased",
                                                      num_labels=len(label_dict),
                                                      output_attentions=False,
                                                      output_hidden_states=False)


device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model.to(device)

model.load_state_dict(torch.load('finetuned_BERT_epoch_4.model', map_location=torch.device('cpu')))

_, predictions, true_vals = evaluate(dataloader_validation)
total_accuracy(predictions, true_vals)

Some weights of the model checkpoint at dccuchile/bert-base-spanish-wwm-uncased were not used when initializing BertForSequenceClassification: ['cls.predictions.decoder.weight', 'cls.predictions.decoder.bias', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.dense.weight']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at dccuc

0.851985559566787

In [83]:
#Predicción

torch.cuda.empty_cache()


model = BertForSequenceClassification.from_pretrained("dccuchile/bert-base-spanish-wwm-uncased",
                                                      num_labels=len(label_dict),
                                                      output_attentions=False,
                                                      output_hidden_states=False)


device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model.to(device)

model.load_state_dict(torch.load('finetuned_BERT_epoch_4.model', map_location=torch.device('cpu')))



Some weights of the model checkpoint at dccuchile/bert-base-spanish-wwm-uncased were not used when initializing BertForSequenceClassification: ['cls.predictions.decoder.weight', 'cls.predictions.decoder.bias', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.dense.weight']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at dccuc

<All keys matched successfully>

In [84]:
model.eval()
predictions = []
    
for batch in dataloader_submission:
  
  batch = tuple(b.to(device) for b in batch)

  inputs = {'input_ids':      batch[0],
                  'attention_mask': batch[1],
                 }

  with torch.no_grad():
    outputs = model(**inputs)
    
    logits = outputs[0]
    logits = logits.detach().cpu().numpy()
    
    
    predictions.append(logits)
    
    


predictions = np.concatenate(predictions, axis=0)
            

In [85]:
submission_prediction=np.argmax(predictions, axis=1).flatten()

label_dict_reverse={
    0:"Petro",
    1:"Uribe",
    2:"Lopez"



}

submission_prediction_df=pd.DataFrame(
    {   
        "id":df_submission["id"],
        "name":submission_prediction


    }

    
    )


submission_prediction_df["name"]=submission_prediction_df["name"].replace(label_dict_reverse)

In [86]:
submission_prediction_df.head()

,id,name
0,fc9a5d778d2b94ed54d658bd,Uribe
1,dd6776d392eeea0a97c12810,Petro
2,09f288d58fc9aec0ff794ee2,Lopez
3,7ad6d2caf3f759f9de4c62cc,Petro
4,daa18d4244188a6fdb29a684,Lopez


In [ ]:
submission_prediction_df.to_csv("submission_prediction.csv",index=False)